In [11]:
import pandas as pd

train_dir='C:\Datasets\labels_and_features_TRAIN.csv'
valid_dir='C:\Datasets\labels_and_features_VAL.csv'

train_data = pd.read_csv(train_dir)
validate_data = pd.read_csv(valid_dir)
X_train = train_data.drop(columns=['teacherID', 'gaze0', 'gaze1', 'gaze2', 'gaze','imgID'])
y_train = train_data['gaze']
X_validate = validate_data.drop(columns=['teacherID', 'gaze0', 'gaze1', 'gaze2', 'gaze','imgID'])
y_validate = validate_data['gaze']
X_train

,distance0,distance1,distance2,relaxation0,relaxation1,relaxation2,gesture0,gesture1,gesture2,posX,...,left_ankleY,left_ankleZ,right_ankleX,right_ankleY,right_ankleZ,bb_x1,bb_y1,bb_x2,bb_y2,bb_score
0,6858,6069,3057,2184,2550,1874,0,0,0,574,...,389.38,0.046153,555.64,388.12,0.032613,538.48,252.53,614.34,381.45,0.94493
1,6154,7625,4634,7702,5500,3507,0,0,0,667,...,368.72,0.153880,672.11,356.37,0.088858,629.56,266.82,715.78,368.46,0.97231
2,6942,7650,4549,6295,6300,3254,0,0,0,188,...,416.04,0.125830,135.74,407.69,0.119140,130.42,244.29,215.40,399.65,0.95616
3,5731,4100,3564,2775,2600,2804,0,0,0,910,...,329.69,0.084865,893.06,329.69,0.083568,862.76,137.36,950.53,311.34,0.96937
4,7759,7125,4099,2775,4024,3479,1255,1050,467,435,...,715.77,0.105560,422.23,704.76,0.081100,380.33,491.82,475.14,717.32,0.99602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2446,4999,4943,2916,3507,3338,3451,6998,6970,6041,659,...,301.56,0.045865,636.51,301.56,0.063991,567.82,136.18,692.24,266.05,0.93697
2447,5900,7600,5309,4127,3950,3226,0,0,0,1029,...,411.79,0.028974,1010.40,481.71,0.027649,989.72,264.78,1083.60,463.65,0.93495
2448,5112,6850,1987,2522,2575,2269,1227,1000,720,655,...,428.92,0.050064,667.75,371.86,0.034746,600.83,202.29,700.63,407.30,0.98055
2449,7027,6210,4211,5140,3789,2944,0,0,0,655,...,508.26,0.106240,675.36,509.61,0.075479,613.28,373.14,690.00,511.92,0.92757


In [16]:
num_classes = train_data['gaze'].nunique()
print("Number of classes:", num_classes)
y_train

Number of classes: 4


0       3
1       2
2       3
3       3
4       3
       ..
2446    3
2447    3
2448    3
2449    2
2450    2
Name: gaze, Length: 2451, dtype: int64

In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_validate = label_encoder.transform(y_validate)

import lightgbm as lgb

d_train = lgb.Dataset(X_train, label=y_train)
params = {
    'objective': 'multiclass',
    'num_class': 4, 
    # Other parameters
}
model = lgb.train(params, d_train)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002407 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17631
[LightGBM] [Info] Number of data points in the train set: 2451, number of used features: 70
[LightGBM] [Info] Start training from score -3.527585
[LightGBM] [Info] Start training from score -1.169618
[LightGBM] [Info] Start training from score -0.461472
[LightGBM] [Info] Start training from score -3.513792


In [23]:
y_pred = model.predict(X_validate)
y_pred_labels = y_pred.argmax(axis=1)  # Get the predicted labels
original_labels = label_encoder.inverse_transform(y_pred_labels)


In [25]:
from sklearn.metrics import classification_report

report = classification_report(y_validate, y_pred_labels)
print('Accuracy Report:\n', report)


Accuracy Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.85      0.68      0.75       201
           2       0.80      0.96      0.87       370
           3       0.00      0.00      0.00        22

    accuracy                           0.81       605
   macro avg       0.41      0.41      0.41       605
weighted avg       0.77      0.81      0.78       605



C:\Users\isxzl\anaconda3\envs\engagement\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isxzl\anaconda3\envs\engagement\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isxzl\anaconda3\envs\engagement\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
original_labels

array([3, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2, 2, 3,
       2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3, 3,
       3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3,
       2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 2, 3, 2, 3, 2, 3, 3, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3,
       2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 2, 2, 3, 3,
       3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3,
       2, 2, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3,
       3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 2, 3, 2,
       3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 2, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3,
       3, 2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 2, 3,